In [ ]:
import os
import csv
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from scipy import stats
from scipy.stats import pearsonr
from matplotlib.ticker import MultipleLocator
import matplotlib.ticker as ticker
from textwrap import wrap
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
targets = pd.read_csv('/Cog-Ment/R/g_factor_5_folds/target.csv')
targets.columns

In [ ]:
targets.columns = ['eid', '(log)Reaction time', 'Fluid intelligence score',
       'Numeric memory: Maximum digits remembered correctly',
       '(log)Trail making test: Duration to complete numeric path',
       '(log)Trail making test: Duration to complete alphabetic path',
       'Symbol digit substitution: Number of correct matches',
       'Paired associate learning: Number of correct pairs',
       'Tower rearranging: Number of puzzles correct',
       'Matrix pattern completion: Number of puzzles correct',
       '(logx+1)Pairs matching: Incorrect matches',
       'Picture vocabulary: Specific cognitive ability',
       'Prospective memory: Initial answer']

## Plot the distribution of cognitive test scores

In [ ]:
# Cognitive test score distribution
warnings.simplefilter(action='ignore', category=FutureWarning)

fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(35, 20))
columns = targets.drop(columns='eid').columns
axes = axes.flatten()
for i, col in enumerate(columns):
    column_data = targets.drop(columns='eid')[col]

    binwidth = 0.03 if col != 'Matrix pattern completion: Number of puzzles correct' and col != 'Fluid intelligence score' and col != 'Numeric memory: Maximum digits remembered correctly' and col != 'Paired associate learning: Number of correct pairs' and col != 'Tower rearranging: Number of puzzles correct' and col != 'Symbol digit substitution: Number of correct matches' else 0.5
    sns.histplot(column_data, ax=axes[i], color='wheat', binwidth=binwidth, linewidth=0.5)
    axes[i].tick_params(axis='x', labelsize=12)

    
    wrapped_col = '\n'.join(wrap(col, width=27)) # Wrap the column label at spaces to keep words uninterrupted

    axes[i].set_xlabel(wrapped_col, fontsize=20)
    axes[i].set_ylabel(None)
    axes[i].tick_params(axis='y', labelsize=12)
    axes[i].tick_params(axis='x', labelsize=12)

    axes[i].xaxis.set_major_locator(ticker.MultipleLocator(1))

    if col == 'Picture vocabulary: Specific cognitive ability':
        axes[i].xaxis.set_major_locator(ticker.MultipleLocator(0.1))
    elif col in ['(log)Trail making test: Duration to complete alphanumeric path', '(log)Trail making test: Duration to complete numeric path', '(log)Reaction time', '(log1p)Pairs matching: Incorrect matches']:
        axes[i].xaxis.set_major_locator(ticker.MultipleLocator(0.5))
    else:
        axes[i].xaxis.set_major_locator(ticker.MultipleLocator(1))

    #axes[i].yaxis.set_major_locator(locator) # y axis tick interval

    axes[i].set_xlim(left=0) if col != '(log)Reaction time' and col != '(log)Trail making test: Duration to complete numeric path' and col !='(log)Trail making test: Duration to complete alphanumeric path' else axes[i].set_xlim(left=column_data.min()) 

plt.suptitle('Distribution of Cognitive Test Scores', fontsize=40, y=1.0)
plt.subplots_adjust(hspace=0.6, wspace=0.5)
plt.tight_layout()
plt.show()

## Plot the correlation between cognitive test scores

In [ ]:
# Correlation plot: Full
plt.figure(figsize=(10, 10))
correlation_target = targets.drop(columns='eid').corr(method='pearson')
plot = sns.heatmap(correlation_target, cmap='coolwarm', fmt=".2f",
                   xticklabels=correlation_target.columns,
                   yticklabels=correlation_target.columns) #annot=True
plot.set_xticklabels(plot.get_yticklabels(), rotation = 70, ha='right', fontsize = 20)
plt.tick_params(labelsize=20)
#plt.title('Correlations between Cognitive Test Scores', fontsize=25, y=1.05)

# Set the font size for the colorbar
cbar = plot.collections[0].colorbar
cbar.ax.tick_params(labelsize=20)
plt.show()

In [ ]:
# Correlation plot: Lower triangular part
correlation_matrix = targets.drop(columns='eid').corr(method='pearson')
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool), k=1)
plt.figure(figsize=(10, 10))
plot = sns.heatmap(correlation_matrix, cmap='coolwarm', fmt=".2f",
                   xticklabels=correlation_matrix.columns,
                   yticklabels=correlation_matrix.columns,
                   mask=mask) # Apply mask
plot.set_xticklabels(plot.get_xticklabels(), rotation=70, ha='right', fontsize=20)
plt.tick_params(labelsize=20)
cbar = plot.collections[0].colorbar
cbar.ax.tick_params(labelsize=20)

plt.show()

## Compute correlations between cognitive test scores

In [ ]:
# Correlations between test scores
correlation_matrix = targets.drop(columns='eid').corr(method='pearson')
# Create a mask for the upper triangle
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool), k=1)
# Apply the mask to get the lower triangle of the correlation matrix
lower_triangle = pd.DataFrame(correlation_matrix.mask(mask)) #or lower_triangle = correlation_matrix.where(~mask)
column_names = ['(log)RT', 'FIS', 'NM', '(log)TMT-numeric', '(log)TMT-alphabetic', 'SDS', 'PAL', 'Tower Rear.', 'Matrix PC', '(log1p)Pairs Match.', 'Picture Vocab.', 'Prospective Mem.']
lower_triangle.columns = column_names
lower_triangle.index = column_names
lower_triangle.round(2)

In [ ]:
# Extract p-values
p_values = pd.DataFrame(index=column_names, columns=column_names)
for i in range(targets.drop(columns='eid').shape[1]):
    for j in range(i+1, targets.drop(columns='eid').shape[1]):
        _, p_value = pearsonr(targets.drop(columns='eid').iloc[:, i], targets.drop(columns='eid').iloc[:, j])
        p_values.iloc[i, j] = p_value
        p_values.iloc[j, i] = p_value 

mask = np.triu(np.ones_like(p_values, dtype=bool), k=1)
lower_triangle_p_values = p_values.mask(mask)

for col in lower_triangle_p_values.columns:
    lower_triangle_p_values[col] = lower_triangle_p_values[col].apply(lambda x: '{:.3f}'.format(x) if pd.notnull(x) else x)
lower_triangle_p_values

# Split the whole dataset into 5 folds

In [ ]:
kf = KFold(n_splits = 5, shuffle = True, random_state = 0)
# Split the data set into train and test, using random folds, and save datasets and IDs for each split
directory = "/UK_BB/txt_no_colnames/"
# Use a for loop to save each train and test set for each fold
for fold, (train_index, test_index) in enumerate(kf.split(np.array(targets))):
    print("TRAIN:", train_index, "TEST:", test_index)

    # Split the data into train and test sets
    y_train, y_test = np.array(targets)[train_index], np.array(targets)[test_index]
    # Convert to a data frame
    y_train_df, y_test_df = pd.DataFrame(y_train, columns = targets.columns).drop(columns=['eid']), pd.DataFrame(y_test, columns = targets.columns).drop(columns=['eid'])
    # Save
    y_train_df.to_csv(f'/UK_BB/fold_{fold}/train_fold_{fold}.csv', index=False)
    y_test_df.to_csv(f'/UK_BB/fold_{fold}/test_fold_{fold}.csv', index=False)


    # Scale
    scaler = StandardScaler()
    y_train_scaled = scaler.fit_transform(y_train)
    y_test_scaled = scaler.transform(y_test)
    # Convert to a data frame
    y_train_scaled_df, y_test_scaled_df = pd.DataFrame(y_train_scaled, columns = targets.columns).drop(columns=['eid']), pd.DataFrame(y_test_scaled, columns = targets.columns).drop(columns=['eid'])
    # Save
    y_train_scaled_df.to_csv(f'/UK_BB/fold_{fold}/train_scaled_fold_{fold}.csv', index=False)
    y_test_scaled_df.to_csv(f'/UK_BB/fold_{fold}/test_scaled_fold_{fold}.csv', index=False)


    # Extract the 'eid' column for each train and test set and save as csv
    y_train_id = y_train[:, 0]
    y_test_id = y_test[:, 0]
    # Convert to a data frame
    y_train_id_df, y_test_id_df = pd.DataFrame(y_train_id, columns = ['eid']), pd.DataFrame(y_test_id, columns = ['eid'])
    # Save
    y_train_id_df.to_csv(f'/UK_BB/fold_{fold}/train_id_fold_{fold}.csv', index=False)
    y_test_id_df.to_csv(f'/UK_BB/fold_{fold}/test_id_fold_{fold}.csv', index=False)
    
    
    #1 Create the file name for each train and test set using the fold number
    train_file = "train_" + str(fold) + ".csv"
    test_file = "test_" + str(fold) + ".csv"
    # Use the os.path.join function to create the full file path for each file
    train_path = os.path.join(directory, train_file)
    test_path = os.path.join(directory, test_file)
    # Save the train and test sets as txt
    np.savetxt(train_path, y_train, delimiter=",")
    np.savetxt(test_path, y_test, delimiter=",")

    #2 Create the file name for each train and test set using the fold number
    train_file_scaled = "train_scaled_" + str(fold) + ".csv"
    test_file_scaled = "test_scaled_" + str(fold) + ".csv"
    # Use the os.path.join function to create the full file path for each file
    train_path_scaled = os.path.join(directory, train_file_scaled)
    test_path_scaled = os.path.join(directory, test_file_scaled)
    # Save scaled files as txt
    np.savetxt(train_path_scaled, y_train_scaled, delimiter=",")
    np.savetxt(test_path_scaled, y_test_scaled, delimiter=",")

    #3 Create the file name for each train and test set using the fold number
    train_id = "train_id_" + str(fold) + ".csv"
    test_id = "test_is_" + str(fold) + ".csv"
    # Use the os.path.join function to create the full file path for each file
    train_path_id = os.path.join(directory, train_id)
    test_path_id = os.path.join(directory, test_id)
    # Save ids as txt
    np.savetxt(train_path_id, y_train_id)
    np.savetxt(test_path_id, y_test_id)

# Demographics

In [ ]:
# Upload demographics variables
basic_confounds_min = pd.read_csv('/UK_BB/basic_confounds_min.csv').drop(columns='Date of attending assessment centre')
participant_age = basic_confounds_min[['eid', 'Sex', 'Age when attended assessment centre']].merge(targets['eid'])
print(basic_confounds_min.columns)
print(participant_age.columns)

0 = Female;
1 = Male

In [ ]:
print('Age when attended assessment centre, mean:', participant_age['Age when attended assessment centre'].mean().round(2), 'SD:', f"{participant_age['Age when attended assessment centre'].std():.3f}")
print('Age max range:', participant_age['Age when attended assessment centre'].max())
print('Age min range:', participant_age['Age when attended assessment centre'].min())
print('Proportion of males:', (participant_age['Sex'].value_counts()[1] / len(participant_age['Sex']) * 100).round(2))
print('Proportion of females:', (participant_age['Sex'].value_counts()[0] / len(participant_age['Sex']) * 100).round(2))

In [ ]:
# Get mean demographic data for each fold
folds = ["0", "1", "2", "3", "4"]
for fold in folds:
    
    y_train = pd.read_csv(f'/UK_BB/fold_{fold}/train_id_fold_{fold}.csv')
    y_test = pd.read_csv(f'/UK_BB/fold_{fold}/test_id_fold_{fold}.csv')

    conf_train, conf_test = pd.merge(participant_age, y_train['eid'], on='eid'), pd.merge(participant_age, y_test['eid'], on='eid')
    print('TRAINING SET')
    print(f'Number of participants in fold {fold}:', conf_train.shape)
    print(f'Mean age in fold {fold}:', conf_train['Age when attended assessment centre'].mean().round(2), f'SD for age in fold {fold}:', '{0:.3f}'.format(conf_train['Age when attended assessment centre'].std()))
    print(f'Proportion of females in fold {fold}:',  (conf_train['Sex'].value_counts()[0] / len(conf_train['Sex']) * 100).round(2))

    print('TEST SET')
    print(f'Number of participants in fold {fold}:', conf_test.shape)
    print(f'Mean age in fold {fold}:', conf_test['Age when attended assessment centre'].mean().round(2), f'SD for age in fold {fold}:', '{0:.3f}'.format(conf_test['Age when attended assessment centre'].std()))
    print(f'Proportion of females in fold {fold}:',  (conf_test['Sex'].value_counts()[0] / len(conf_test['Sex']) * 100).round(2))

In [ ]:
# Create a table with demographics data for each fold
results_list = []

folds = ["0", "1", "2", "3", "4"]
for fold in folds:
    
    y_train = pd.read_csv(f'/UK_BB/fold_{fold}/train_id_fold_{fold}.csv')
    y_test = pd.read_csv(f'/UK_BB/fold_{fold}/test_id_fold_{fold}.csv')

    conf_train, conf_test = pd.merge(participant_age, y_train['eid'], on='eid'), pd.merge(participant_age, y_test['eid'], on='eid')
    train_results = {
        'Fold': fold,
        'Set': 'Training',
        'Number of Participants': conf_train.shape[0],
        'Mean Age': conf_train['Age when attended assessment centre'].mean().round(2),
        'Age SD': '{0:.3f}'.format(conf_train['Age when attended assessment centre'].std()),
        'Proportion of Females (%)': (conf_train['Sex'].value_counts()[0] / len(conf_train['Sex']) * 100).round(2)
    }
    results_list.append(train_results)
    
    # Store test set results in a dictionary and append to the list
    test_results = {
        'Fold': fold,
        'Set': 'Test',
        'Number of Participants': conf_test.shape[0],
        'Mean Age': conf_test['Age when attended assessment centre'].mean().round(2),
        'Age SD': '{0:.3f}'.format(conf_test['Age when attended assessment centre'].std()),
        'Proportion of Females (%)': (conf_test['Sex'].value_counts()[0] / len(conf_test['Sex']) * 100).round(2)
    }
    results_list.append(test_results)
results_df = pd.DataFrame(results_list)
results_df.to_csv('/UK_BB/folds_summary.csv', index=False)